In [ ]:
!pip install mediapipe-model-maker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import json
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import object_detector

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
!unzip -q /content/feet_pics.zip


replace feet_pics/train/Annotations/017OW9VKXGQM.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace feet_pics/train/Annotations/09XDHJSQGPI0.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!mkdir /content/cache_dir
!mkdir /content/cache_dir/train_data
!mkdir /content/cache_dir/val_data

mkdir: cannot create directory ‘/content/cache_dir’: File exists
mkdir: cannot create directory ‘/content/cache_dir/train_data’: File exists
mkdir: cannot create directory ‘/content/cache_dir/val_data’: File exists


In [ ]:
train_data = object_detector.Dataset.from_pascal_voc_folder('/content/feet_pics/train',cache_dir="/content/cache_dir/train_data")

In [ ]:
val_data = object_detector.Dataset.from_pascal_voc_folder('/content/feet_pics/val', cache_dir="/content/cache_dir/val_data")

In [ ]:
print("train_data size: ", train_data.size)
print("validation_data size: ", val_data.size)

train_data size:  194
validation_data size:  55


In [ ]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

In [ ]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=val_data,
    options=options)

/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 32)  3704416   
                             , '3': (None, 32, 32, 64            
                             ),                                  
                              '4': (None, 16, 16, 160            
                             ),                                  
                              '5': (None, 8, 8, 192),            
                              '6': (None, 1, 1, 1280)            
                             }                                   
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128),  144928    
                              '4': (None, 16, 16, 128            
                             ),                                  
                              '3': (None, 32, 32, 

/usr/local/lib/python3.10/dist-packages/keras/backend.py:451: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


24/24 [==============================] - 55s 451ms/step - total_loss: 1.9427 - cls_loss: 1.3184 - box_loss: 0.0112 - model_loss: 1.8795 - val_total_loss: 11.9431 - val_cls_loss: 6.6132 - val_box_loss: 0.1053 - val_model_loss: 11.8796
Epoch 2/30
24/24 [==============================] - 4s 186ms/step - total_loss: 1.1892 - cls_loss: 0.7408 - box_loss: 0.0077 - model_loss: 1.1256 - val_total_loss: 2.9712 - val_cls_loss: 1.9833 - val_box_loss: 0.0185 - val_model_loss: 2.9074
Epoch 3/30
24/24 [==============================] - 5s 190ms/step - total_loss: 0.8885 - cls_loss: 0.5322 - box_loss: 0.0058 - model_loss: 0.8247 - val_total_loss: 2.9578 - val_cls_loss: 2.2060 - val_box_loss: 0.0138 - val_model_loss: 2.8940
Epoch 4/30
24/24 [==============================] - 4s 180ms/step - total_loss: 0.7052 - cls_loss: 0.4025 - box_loss: 0.0048 - model_loss: 0.6414 - val_total_loss: 1.3244 - val_cls_loss: 0.8309 - val_box_loss: 0.0086 - val_model_loss: 1.2605
Epoch 5/30
24/24 [======================

In [ ]:
loss, coco_metrics = model.evaluate(val_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

14/14 [==============================] - 2s 56ms/step - total_loss: 0.9990 - cls_loss: 0.5904 - box_loss: 0.0069 - model_loss: 0.9348
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.37s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.216
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.570
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.127
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.193
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.275
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.200
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.378
 Average Recall     (AR) @[ 

In [ ]:
from google.colab import files
model.export_model('feetdetect.tflite')
!ls -lh exported_model
files.download('exported_model/feetdetect.tflite')

Exporting a floating point model


/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


total 30M
-rw-r--r-- 1 root root  13M May 24 17:31 feetdetect.tflite
-rw-r--r-- 1 root root  16M May 24 17:29 float_ckpt.data-00000-of-00001
-rw-r--r-- 1 root root  32K May 24 17:29 float_ckpt.index
-rw-r--r-- 1 root root 2.4M May 24 17:31 metadata.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
qat_hparams = object_detector.QATHParams(learning_rate=0.3, batch_size=4, epochs=10, decay_steps=6, decay_rate=0.96)
model.quantization_aware_training(train_data, val_data, qat_hparams=qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(val_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")

Epoch 1/10


48/48 [==============================] - 98s 730ms/step - total_loss: 2.6463 - cls_loss: 1.2314 - box_loss: 0.0238 - model_loss: 2.4196 - val_total_loss: 1.8053 - val_cls_loss: 0.8941 - val_box_loss: 0.0123 - val_model_loss: 1.5099
Epoch 2/10
48/48 [==============================] - 26s 536ms/step - total_loss: 1.8933 - cls_loss: 0.9102 - box_loss: 0.0137 - model_loss: 1.5955 - val_total_loss: 1.7982 - val_cls_loss: 0.8831 - val_box_loss: 0.0123 - val_model_loss: 1.4997
Epoch 3/10
48/48 [==============================] - 26s 539ms/step - total_loss: 1.6485 - cls_loss: 0.7642 - box_loss: 0.0117 - model_loss: 1.3500 - val_total_loss: 1.7127 - val_cls_loss: 0.7992 - val_box_loss: 0.0123 - val_model_loss: 1.4142
Epoch 4/10
48/48 [==============================] - 26s 550ms/step - total_loss: 1.5291 - cls_loss: 0.6826 - box_loss: 0.0110 - model_loss: 1.2306 - val_total_loss: 1.7385 - val_cls_loss: 0.8029 - val_box_loss: 0.0127 - val_model_loss: 1.4400
Epoch 5/10
48/48 [=====================

In [ ]:
new_qat_hparams = object_detector.QATHParams(learning_rate=0.9, batch_size=4, epochs=15, decay_steps=5, decay_rate=0.96)
model.restore_float_ckpt()
model.quantization_aware_training(train_data, val_data, qat_hparams=new_qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(val_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")

/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Using existing files at /tmp/model_maker/object_detector/mobilenetmultiavg
Model: "retina_net_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net_2 (MobileNet)    {'2': (None, 64, 64, 32)  3704416   
                             , '3': (None, 32, 32, 64            
                             ),                                  
                              '4': (None, 16, 16, 160            
                             ),                                  
                              '5': (None, 8, 8, 192),            
                              '6': (None, 1, 1, 1280)            
                             }                                   
                                                                 
 fpn_2 (FPN)                 {'5': (None, 8, 8, 128),  144928    
                              '4': (None, 16, 16, 128            
                             ),        

/usr/local/lib/python3.10/dist-packages/keras/backend.py:451: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


48/48 [==============================] - 100s 780ms/step - total_loss: 1.9955 - cls_loss: 0.9124 - box_loss: 0.0183 - model_loss: 1.8268 - val_total_loss: 2.3048 - val_cls_loss: 1.2168 - val_box_loss: 0.0175 - val_model_loss: 2.0920
Epoch 2/15
48/48 [==============================] - 26s 541ms/step - total_loss: 1.3568 - cls_loss: 0.6193 - box_loss: 0.0105 - model_loss: 1.1426 - val_total_loss: 2.2262 - val_cls_loss: 1.1959 - val_box_loss: 0.0163 - val_model_loss: 2.0115
Epoch 3/15
48/48 [==============================] - 27s 557ms/step - total_loss: 1.2136 - cls_loss: 0.5374 - box_loss: 0.0092 - model_loss: 0.9989 - val_total_loss: 1.9484 - val_cls_loss: 0.9823 - val_box_loss: 0.0150 - val_model_loss: 1.7337
Epoch 4/15
48/48 [==============================] - 26s 540ms/step - total_loss: 1.1833 - cls_loss: 0.5284 - box_loss: 0.0088 - model_loss: 0.9686 - val_total_loss: 1.7380 - val_cls_loss: 0.8641 - val_box_loss: 0.0132 - val_model_loss: 1.5234
Epoch 5/15
48/48 [====================

In [ ]:
from google.colab import files
model.export_model('ftdetect.tflite')
!ls -lh exported_model
files.download('exported_model/ftdetect.tflite')

Exporting a qat int8 model


/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


total 22M
-rw-r--r-- 1 root root  16M May 24 10:12 float_ckpt.data-00000-of-00001
-rw-r--r-- 1 root root  32K May 24 10:12 float_ckpt.index
-rw-r--r-- 1 root root 3.7M May 24 10:42 ftdetect.tflite
-rw-r--r-- 1 root root 2.4M May 24 10:42 metadata.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>